In [20]:
%%writefile deploy/score.py
import json, os
from io import BytesIO
import base64
from dalle_model import DalleModel

def init(): 
  global dalle_model
  dalle_model = DalleModel('Mega_full')
  dalle_model.generate_images("CN tower is flying to Mars", 1)

def run(data): 
  print(data)
  input_data = json.loads(data)
  # print(input_data)
  text_prompt = input_data["text"]
  num_images = input_data["num_images"]
  generated_imgs = dalle_model.generate_images(text_prompt, num_images)

  returned_generated_images = []
  
  for idx, img in enumerate(generated_imgs):
      buffered = BytesIO()
      img.save(buffered, format='jpeg')
      img_str = base64.b64encode(buffered.getvalue()).decode("utf-8")
      returned_generated_images.append(img_str)

  print(f"Created {num_images} images from text prompt [{text_prompt}]")
  
  response = {'generatedImgs': returned_generated_images}
  return json.dumps(response)

Overwriting deploy/score.py


In [21]:
from azureml.core.model import InferenceConfig, Model
from azureml.core import Environment, Workspace
from azureml.core.webservice import LocalWebservice, AksWebservice, AciWebservice

workspace = Workspace.from_config()

service_name = 'dall-e-endpoint'
env = Environment.get(workspace=workspace, name="dall-e")

inference_config = InferenceConfig(entry_script='score.py', 
                            source_directory='deploy',
                            environment=env)

# deploy to AKS cluster
deployment_config =  AksWebservice.deploy_configuration(cpu_cores = 4,
                                                        memory_gb= 28,
                                                        gpu_cores = 1, 
                                                        compute_target_name='gpu-aks-cluster')

service = Model.deploy(
    workspace,
    name = service_name,
    models=[],
    inference_config= inference_config,
    deployment_config= deployment_config,
    overwrite=True,
)
service.wait_for_deployment(show_output=True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2022-10-03 12:14:53+00:00 Creating Container Registry if not exists.
2022-10-03 12:14:53+00:00 Registering the environment.
2022-10-03 12:14:54+00:00 Use the existing image.
2022-10-03 12:14:58+00:00 Checking the status of deployment dall-e-endpoint..
2022-10-03 12:17:38+00:00 Checking the status of inference endpoint dall-e-endpoint.
Succeeded
AKS service creation operation finished, operation "Succeeded"


## Sample code to run DALL-E locally
``` python
from dalle_model import DalleModel
dalle_model = DalleModel('Mega_full')
generated_imgs = dalle_model.generate_images("CN tower is landing to the moon", 1)

returned_generated_images = []
dir_name = os.path.join('images')
# Path(dir_name).mkdir(parents=True, exist_ok=True)

for idx, img in enumerate(generated_imgs):
    img.save(os.path.join(dir_name, f'{idx}.jpg'), format='jpeg')
```